## Model Evaluation

In [1]:
import os

In [2]:
%pwd

'/home/utpal108/dev/Python/PW_Skills_Projects/plant-disease-detection/notebooks'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/home/utpal108/dev/Python/PW_Skills_Projects/plant-disease-detection'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int
    evaluation_score_path: Path

In [6]:
from plantDiseaseDetection.constants import *
from plantDiseaseDetection.utils import create_directories, read_yaml, save_json

In [7]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_validation_config(self) -> EvaluationConfig:
        
        eval_config = EvaluationConfig(
            path_of_model =  Path(self.config.training.trained_model_path),
            training_data = Path(self.config.data_ingestion.dataset_dir),
            all_params = self.params,
            params_image_size = self.params.IMAGE_SIZE,
            params_batch_size = self.params.BATCH_SIZE,
            evaluation_score_path = Path(self.config.evaluation.evaluation_score_path)
        )
        
        return eval_config

In [8]:
import tensorflow as tf

2023-11-14 15:58:11.694121: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-14 15:58:11.732908: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-14 15:58:11.732948: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-14 15:58:11.732985: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-14 15:58:11.740881: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-14 15:58:11.741634: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [27]:
class Evaluate:

    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):
        
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.30
        )

        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = 'bilinear'
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            subset = 'validation',
            shuffle = False,
            **dataflow_kwargs
        )

    def load_model(self, path:Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        score = model.evaluate(self.valid_generator)
        save_json(path = self.config.evaluation_score_path, data={'loss': score[0], 'accuracy': score[1]})


In [28]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = Evaluate(config=val_config)
    evaluation.evaluation()

except Exception as e:
    raise e

2023-11-14 21:55:00,855 : plantDiseaseDetection.logger.logger_config - INFO - YAML file: config/config.yaml loaded successfully
2023-11-14 21:55:00,859 : plantDiseaseDetection.logger.logger_config - INFO - YAML file: params.yaml loaded successfully
2023-11-14 21:55:00,861 : plantDiseaseDetection.logger.logger_config - INFO - created directory at: artifacts


Found 144 images belonging to 5 classes.
9/9 [==============================] - 14s 1s/step - loss: 1.6085 - accuracy: 0.2222


2023-11-14 21:55:14,947 : plantDiseaseDetection.logger.logger_config - INFO - json file saved at: artifacts/training/scores.json
